# Comparison of RFS Regression - Manifold vs PCA

### Manifold Learning with UMAP

In [5]:
import pandas as pd
import numpy as np

all_df = pd.read_excel('../TrainDataset2024.xls', index_col=False)
all_df.drop('ID', axis=1, inplace=True)
all_df.shape

import pickle
from sklearn.impute import SimpleImputer

import json
keep_feat_names = []
with open('../gene_clf_selected_features.json', 'rb') as f:
  keep_feat_names = json.load(f)
  
# replace missing gene with classification result
# see train_gene_classifier.ipynb for more details
with open('../log_reg_gene_classifier.pkl', 'rb') as f:
  log_res_clf = pickle.load(f)
  
  # rebuild prediction df
  gene_impute_df = all_df.copy()

  temp_X = gene_impute_df.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)
  y = gene_impute_df['Gene']

  print("before impute:") 
  print(gene_impute_df['Gene'].value_counts())

  keep_df = temp_X[keep_feat_names]
  replace_index = keep_df[keep_df['Gene'] == 999].index

  # get prediction on missing gene
  target = gene_impute_df.loc[replace_index, keep_feat_names]
  target.drop('Gene', axis=1, inplace=True)

  pred = log_res_clf.predict(target)
  gene_impute_df.loc[replace_index, 'Gene'] = pred

  print("after impute:") 
  print(gene_impute_df['Gene'].value_counts())

  # assign back to all_df
  all_df['Gene'] = gene_impute_df['Gene']

# Replace missing values with median of the column
imputer = SimpleImputer(strategy="median", missing_values=999)
all_df[:] = imputer.fit_transform(all_df)

# classification target
clf_y = all_df['pCR (outcome)']
# regression target
rgr_y = all_df['RelapseFreeSurvival (outcome)']

all_df.shape

before impute:
Gene
0      193
1      119
999     88
Name: count, dtype: int64
after impute:
Gene
0    281
1    119
Name: count, dtype: int64


/Users/stephen/Library/Caches/pypoetry/virtualenvs/labs-env-NHQiDXzZ-py3.12/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


(400, 120)

### Outlier Removal

In [6]:
# Outlier removal approach by:
# Thanaki, Jalaj. Machine Learning Solutions : Expert Techniques to Tackle Complex Machine Learning Problems Using Python, Packt Publishing, Limited, 2018. 
# ProQuest Ebook Central, Available at: http://ebookcentral.proquest.com/lib/nottingham/detail.action?docID=5379696.

# Outlier detection using the following methods:
# 1. Percentile based outlier detection
# 2. MAD (median absolute deviation) based outlier detection
# 3. Standard deviation based outlier detection

""" 
    Get all the data points that lie under the percentile range from 2.5 to 97.5
"""
def percentile_based_outlier(data, threshold=95):
    diff = (100 - threshold) / 2.0
    minval, maxval = np.percentile(data, [diff, 100 - diff])
    return (data < minval) | (data > maxval)

"""
    Get all the data points that lie under a threshold of 3.5 using modified Z-score (based on the median absolute deviation)
"""
def mad_based_outlier(points, threshold=3.5):
    points = np.array(points)
    if len(points.shape) == 1:
        points = points[:, None]
    median_y = np.median(points)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in points])
    # Small constant added to avoid division by zero
    modified_z_scores = [0.6745 * (y - median_y) / (median_absolute_deviation_y + 1e-6) for y in points]

    return np.abs(modified_z_scores) > threshold

"""
    Get all the data points that lie under a threshold of 3 using standard deviation
"""
def std_div(data, threshold=3):
    std = data.std()
    mean = data.mean()
    isOutlier = []
    for val in data:
        if abs(val - mean)/std > threshold:
            isOutlier.append(True)
        else:
            isOutlier.append(False)
    return isOutlier

"""
    Perform an outlier voting system to determine if a data point is an outlier. 
    If two of the three methods agree that a data point is an outlier, then it is removed.
"""
def outlierVote(data):
    x = percentile_based_outlier(data)
    y = mad_based_outlier(data)
    z = std_div(data)
    temp = list(zip(x, y, z))
    final = []
    for i in range(len(temp)):
        if temp[i].count(False) >= 2:
            final.append(False)
        else:
            final.append(True)
    return final

def removeOutliers(data):
    # Remove outliers from the dataframe
    for column in data.columns:
        outliers = outlierVote(all_df[column])
        # Calculate Non-Outlier Maximum and minimum using the outliers list
        non_outlier_max = all_df.loc[~np.array(outliers), column].max()
        non_outlier_min = all_df.loc[~np.array(outliers), column].min()

        # Replace outliers with the maximum or minimum non-outlier value
        for i, outlier in enumerate(outliers):
            if outlier:
                data.loc[i, column] = non_outlier_max if data.loc[i, column] > non_outlier_max else non_outlier_min

# Remove outliers, assign modified features to X and drop the outcome columns
removeOutliers(all_df)
X = all_df.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)

In [7]:
from umap import UMAP
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xs_non_mri = scaler.fit_transform(X.iloc[:, :11])

umap = UMAP(n_components=2, random_state=42)
X_umap_mri = umap.fit_transform(X.iloc[:, 11:])

Xs = np.c_[Xs_non_mri, X_umap_mri]

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne_mri = tsne.fit_transform(X.iloc[:,11:])
standard_scaler = StandardScaler()
Xs_non_mri = standard_scaler.fit_transform(X.iloc[:,0:11])
Xs = np.c_[Xs_non_mri, X_tsne_mri]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs, rgr_y, test_size=0.2, random_state=42)

# svr 
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 30, 45, 50, 75, 80, 85, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

grid_search = GridSearchCV(SVR(), param_grid, n_jobs=-1, cv=5, verbose=1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# validation 
from sklearn.metrics import mean_absolute_error, r2_score

best = grid_search.best_estimator_

y_pred = best.predict(X_test)
print(y_pred[:10])

print("MAE: ", mean_absolute_error(y_test, y_pred))
print("R2: ", r2_score(y_test, y_pred))

/Users/stephen/Library/Caches/pypoetry/virtualenvs/labs-env-NHQiDXzZ-py3.12/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[54.43768826 52.34407289 54.43788576 54.43851049 54.43794485 54.49990334
 54.44435872 53.84006723 54.45210805 54.44891427]
MAE:  21.4910116113379
R2:  -0.017211227922942385


### Part 2 - PCA on normally distributed MRI features + top 15 Best features

In [8]:
import pandas as pd
import numpy as np

all_df = pd.read_excel('../TrainDataset2024.xls', index_col=False)
all_df.drop('ID', axis=1, inplace=True)
all_df.head()

'''
Strategy: 
We trained a logistic regression model to predict the missing gene values.
see the training for this model in train_gene_classifier.ipynb

steps:
1. Imputer missing Gene with gene_classifier predictions
2. Impute missing values in all other columns with median 
'''

import pickle
from sklearn.impute import SimpleImputer

'''
Load the features that were used in the training of the gene classifier.

The gene_clf_selected_features.json file is created in train_gene_classifier.ipynb
'''
import json
keep_feat_names = []
with open('../gene_clf_selected_features.json', 'rb') as f:
  keep_feat_names = json.load(f)
  
# Step 1. Imputer missing Gene with gene_classifier predictions
# load the gene classifier model
with open('../log_reg_gene_classifier.pkl', 'rb') as f:
  log_res_clf = pickle.load(f)
  
  # Get row indicies of missing gene values
  gene_impute_df = all_df.copy()
  temp_X = gene_impute_df.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)
  y = gene_impute_df['Gene']
  keep_df = temp_X[keep_feat_names]
  replace_index = keep_df[keep_df['Gene'] == 999].index

  print("Before Impute:")
  print(gene_impute_df.iloc[replace_index, :]['Gene'][:3])

  # shape the target data set
  target = gene_impute_df.loc[replace_index, keep_feat_names]
  target.drop('Gene', axis=1, inplace=True)

  # make classification predictions
  pred = log_res_clf.predict(target)
  # replace missing gene values with predictions
  gene_impute_df.loc[replace_index, 'Gene'] = pred

  print("After Impute:") 
  print(gene_impute_df.iloc[replace_index, :]['Gene'][:3])

  # assign the gene values back to the main dataframe
  all_df['Gene'] = gene_impute_df['Gene']

# Step 2. Impute missing values in all other columns with median
# Replace missing values with median of the column
imputer = SimpleImputer(strategy="median", missing_values=999)
all_df[:] = imputer.fit_transform(all_df)

# Save regression target label for later use
y = all_df['RelapseFreeSurvival (outcome)']

removeOutliers(all_df)
X = all_df.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)

from scipy import stats
from sklearn.ensemble import RandomForestRegressor
'''
  Feature Selection and Dimensionality Reduction strategy:
  Combine the following feature categories:
    1. Mandatory features - 'ER', 'HER2', 'Gene'
    2. Select the nomral MRI features and apply PCA
    3. Select the best features from the rest of the features (non MRI)
  Then combine all the selected features and apply PCA to reduce the dimensionality of the final feature set.
'''

X = all_df.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)

# A dictionary for storing columns names used during training for the final test predictions
cache = {} 

# 1. Mandarory feartures
mandatory_features_indices = [1,3,10]
print(f"Mandatory features: {X.columns[mandatory_features_indices]}")

# 2. Normally distributed MRI features and apply PCA
mri_indices = list(range(11, X.shape[1]))
mri_features = X.iloc[:, mri_indices]
alpha = 0.05
normal_cols_idx = []
for idx, col in enumerate(mri_features.columns):
  res = stats.normaltest(mri_features[col])
  if res.pvalue > alpha:
    normal_cols_idx.append(idx)
print(f"Normal MRI features: {list(mri_features.columns[normal_cols_idx])}")

# Apply PCA to the MRI features
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X_mri = X.iloc[:, normal_cols_idx]
X_mri_pca = pca.fit_transform(X_mri)
print(f"Normal MRI PCA shape: {X_mri_pca.shape}")

# save normally distributed MRI features index
cache['normal_cols_idx'] = list(normal_cols_idx)

# 3. Select best features from the rest of the features
# Start by selectinf best features from the all features
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X, y)
importances = reg.feature_importances_
important_features_idx = np.argsort(importances)[::-1][:15]

# Filter out normally distributed MRI features
important_features_idx = [i for i in important_features_idx if i not in normal_cols_idx and i not in mandatory_features_indices]
print(f"Important features: {list(X.columns[important_features_idx])}")
cache['important_features_idx'] = [int(idx) for idx in important_features_idx]

# 4. Build final feature set
# Combine Mandatory and important features indices
final_indices = mandatory_features_indices + important_features_idx
print(f"Selected final features: {len(final_indices)} -  {list(X.columns[final_indices])}")

# Add pca of normal MRI features
Xs = X.iloc[:, final_indices]
Xs = np.concatenate((Xs, X_mri_pca), axis=1)
print(f"Final Shape: {Xs.shape}")

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=42)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
rnd_forest = RandomForestRegressor(random_state=42)

'''
Use GridSearchCV to find the best hyperparameters for the model
'''
param_grid = {
    'max_depth': [1, 2, 4, 7, 11, 50, 100],
    'n_estimators': [50, 75, 100],
    'max_features': ['sqrt'],
    'min_samples_split': [2, 5, 7, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search
grid_search = GridSearchCV(estimator=rnd_forest, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
best_rnd_forest = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")

# Make predictions and evaluate the model
y_pred = best_rnd_forest.predict(X_test)
rnd_mae = mean_absolute_error(y_test, y_pred)
rnd_rmse = root_mean_squared_error(y_test, y_pred)
rnd_r2 = r2_score(y_test, y_pred)
result = {
    'best_params': grid_search.best_params_,
    'mean_absolute_error': rnd_mae,
    'root_mean_squared_error': rnd_rmse,
    'r2_score': rnd_r2
}
print("MAE: ", rnd_mae)
print("R2: ", rnd_r2)

Before Impute:
28     999
123    999
161    999
Name: Gene, dtype: int64
After Impute:
28     0
123    0
161    0
Name: Gene, dtype: int64


/Users/stephen/Library/Caches/pypoetry/virtualenvs/labs-env-NHQiDXzZ-py3.12/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Mandatory features: Index(['ER', 'HER2', 'Gene'], dtype='object')
Normal MRI features: ['original_firstorder_Skewness', 'original_gldm_DependenceEntropy', 'original_glrlm_RunEntropy']
Normal MRI PCA shape: (400, 1)
Important features: ['original_firstorder_Range', 'original_firstorder_Kurtosis', 'original_glszm_ZoneEntropy', 'original_firstorder_90Percentile', 'Age', 'original_glszm_SizeZoneNonUniformity', 'original_firstorder_Skewness', 'original_firstorder_Maximum', 'original_glszm_SizeZoneNonUniformityNormalized', 'original_glszm_ZonePercentage', 'original_shape_Elongation', 'original_firstorder_Variance', 'original_shape_MinorAxisLength', 'original_shape_Flatness', 'original_glszm_SmallAreaHighGrayLevelEmphasis']
Selected final features: 18 -  ['ER', 'HER2', 'Gene', 'original_firstorder_Range', 'original_firstorder_Kurtosis', 'original_glszm_ZoneEntropy', 'original_firstorder_90Percentile', 'Age', 'original_glszm_SizeZoneNonUniformity', 'original_firstorder_Skewness', 'original_fir